In [ ]:
import pandas as pd 
import seaborn as sns
import numpy as np
import  matplotlib.pyplot as plt
import datetime


In [115]:
input_data = pd.read_excel(r'/Users/dylan/Github repo/DataBase/signal_process/benchmark_result-30_5.xlsx',sheet_name = 'Sheet1')
signal_input = pd.read_excel(r'/Users/dylan/Github repo/DataBase/signal_process/benchmark_result-30_5.xlsx',sheet_name = 'Sheet3')
signal_input_roll = signal_input[signal_input['model'] == 'benchmark']


In [ ]:
# 透视 
# input_pivot = input_data.pivot_table(index = 'start_date',columns = 'model',values = 'sharpe_ratio_sp')
# 判断集合
# input_data.drop(input_data.tail(int(len(input_data)/2)).index, inplace=True)

In [116]:
# 测试
# signal_input['start_date'] = pd.to_datetime(signal_input.start_date, format = r'%Y%m%d', errors = 'ignore') 

signal_input_roll.drop(labels = 'Unnamed: 0',axis = 1, inplace = True)
signal_input_roll.reset_index(drop = True)
signal_input.drop(labels = 'Unnamed: 0',axis = 1, inplace = True)
signal_input.reset_index(drop = True)


,start_date,end_date,model,annual_return_sp,sharpe_ratio_sp,win_rate_sp,pc_ratio_sp,sp_rank
0,2018-05-01,2018-06-12,2日开盘价持续下跌-1120-bagc,3.41,3.27,0.41,1.87,0.656716
1,2018-05-01,2018-06-12,2日开盘价持续下跌-1120-dtc,3.41,3.27,0.41,1.87,0.656716
2,2018-05-01,2018-06-12,2日开盘价持续下跌-1120-knnc,-0.28,-0.35,0.25,1.05,0.074627
3,2018-05-01,2018-06-12,2日开盘价持续下跌-1120-lr,3.41,3.27,0.41,1.87,0.656716
4,2018-05-01,2018-06-12,2日开盘价持续下跌-1120-rfc,2.02,1.43,0.40,1.03,0.231343
...,...,...,...,...,...,...,...,...
9643,2021-04-08,2021-05-24,2日开盘价持续下跌-1230-knnc,-1.67,-1.51,0.67,1.78,0.626866
9644,2021-04-08,2021-05-24,2日开盘价持续下跌-1230-lr,-2.18,-1.34,0.00,1.55,0.723881
9645,2021-04-08,2021-05-24,2日开盘价持续下跌-1230-rfc,-6.01,-4.67,0.50,1.33,0.231343
9646,2021-04-08,2021-05-24,2日开盘价持续下跌-1230-xgb,-4.62,-3.12,1.00,1.59,0.462687


In [117]:
# 正确
list_ = list(signal_input_roll.rolling(window = 10,min_periods = 10)['start_date'])[10:]
list_[0]
# signal_input.set_index('start_date',inplace = False).loc[list_[0]]

133   2018-05-09
200   2018-05-16
267   2018-05-23
334   2018-05-30
401   2018-06-06
468   2018-06-13
535   2018-06-21
602   2018-06-28
669   2018-07-05
736   2018-07-12
Name: start_date, dtype: datetime64[ns]

In [95]:
def get_model(x,window):
    def get_value(m):
        value = m['sharpe_ratio_sp'] - m[m['model'] == 'benchmark']['sharpe_ratio_sp'].iat[0] # 返回一个序列
        return value

    def get_lable(n):
        if n >= 0:
            return True
        else: 
            return False
    
    list_ = list(x.rolling(window = window,min_periods = window)['start_date'])[window:]    
    model_list = []
    for i in range(len(list_)):
        signal_input = x.set_index('start_date',inplace = False).loc[list_[i]]
        signal_input['value'] = list(x.groupby('start_date').apply(get_value)) # 转为list
        signal_input['lable'] = list(signal_input['value'].apply(get_lable))
        signal_input.loc[:,'value'] = signal_input.transform({'value' : lambda x : np.square(x)})
        # input_data.groupby(['model','lable']).sum().apply(get_loss)
        input_filter = signal_input.groupby(['model','lable']).agg({'value' : 'sum'})
        input_filter.drop(input_filter.tail(1).index,inplace=True) # 舍去benchmark
        input_filter['loss'] = [input_filter.loc[model].diff()['value'].iat[1] for model in list(input_filter.index.get_level_values(0))]
        model_list  = list(input_filter['loss'].nlargest(10).index.get_level_values(0).drop_duplicates())
    model_list = model_list.append(model_list)

In [210]:
# 正确

# signal_input_roll.drop(labels = 'Unnamed: 0',axis = 1, inplace = True)
signal_input_roll.reset_index(drop = True)
# signal_input.drop(labels = 'Unnamed: 0',axis = 1, inplace = True)
signal_input.reset_index(drop = True)

def get_value(m):
    value = m['sharpe_ratio_sp'] - m[m['model'] == 'benchmark']['sharpe_ratio_sp'].iat[0] # 返回一个序列
    return value

def get_lable(n):
    if n >= 0:
        return True
    else: 
        return False
    
list_ = list(signal_input_roll.rolling(window = 10,min_periods = 10)['start_date'])[10:]    
model_list = pd.DataFrame()
list1 = []
for i in range(len(list_)):
    list1 = list1 + list(list_[i])
    x = signal_input.set_index('start_date',inplace = False).loc[list1]
    x['value'] = list(x.groupby('end_date').apply(get_value)) # 转为list
    x['lable'] = list(x['value'].apply(get_lable))
    x.loc[:,'value'] = x.transform({'value' : lambda x : np.square(x)})
    # input_data.groupby(['model','lable']).sum().apply(get_loss)
    input_filter = x.groupby(['model','lable']).agg({'value' : 'sum'})
    input_filter.drop(input_filter.tail(1).index,inplace=True)
    input_filter = input_filter.unstack(fill_value = 0).loc[:,'value']
    input_filter.loc[:,'loss'] = input_filter[True] - input_filter[False]
    model_filter = input_filter.nlargest(10,'loss',keep = 'first')
    #model_list  = list(input_filter['loss'].nlargest(10).index.get_level_values(0).drop_duplicates())
    model_list = model_list.append(model_filter)
    
# 统计频次
print(len(list1))
pd.Series(model_list.index).value_counts()

1340


2日开盘价持续下跌-1131-rfc     134
2日开盘价持续下跌-1131-xgb     130
2日开盘价持续下跌-1131-knnc    128
2日开盘价持续下跌-1131-dtc     122
2日开盘价持续下跌-1133-bagc     98
2日开盘价持续下跌-1121-rfc      92
2日开盘价持续下跌-1132-knnc     67
2日开盘价持续下跌-1131-lr       65
2日开盘价持续下跌-1133-knnc     59
2日开盘价持续下跌-1122-knnc     58
2日开盘价持续下跌-1222-knnc     55
2日开盘价持续下跌-1122-bagc     43
2日开盘价持续下跌-1132-xgb      36
2日开盘价持续下跌-1120-knnc     30
2日开盘价持续下跌-1122-rfc      29
2日开盘价持续下跌-1132-dtc      27
2日开盘价持续下跌-1221-dtc      25
2日开盘价持续下跌-1130-xgb      25
2日开盘价持续下跌-1120-xgb      25
2日开盘价持续下跌-1121-xgb      14
2日开盘价持续下跌-1221-xgb      11
2日开盘价持续下跌-1222-bagc     10
2日开盘价持续下跌-1221-knnc     10
2日开盘价持续下跌-1132-bagc     10
2日开盘价持续下跌-1121-bagc      7
2日开盘价持续下跌-1122-dtc       6
2日开盘价持续下跌-1122-xgb       6
2日开盘价持续下跌-1121-dtc       6
2日开盘价持续下跌-1132-rfc       5
2日开盘价持续下跌-1133-dtc       5
2日开盘价持续下跌-1130-dtc       1
2日开盘价持续下跌-1121-knnc      1
Name: model, dtype: int64

In [209]:
len(list(set(list1)))

143

In [ ]:
# 更改为链式方法


In [177]:
df = pd.DataFrame({'foo': ['one', 'one', 'one', 'two', 'two',
                           'two'],
                   'bar': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'baz': [1, 2, 3, 4, 5, 6],
                   'zoo': ['x', 'y', 'z', 'q', 'w', 't']})

In [178]:
df.columns

Index(['foo', 'bar', 'baz', 'zoo'], dtype='object')

In [ ]:
def get_loss(window,period):
        def get_value(x):
            value = x['sharpe_ratio_sp'] - x[x['model'] == 'benchmark']['sharpe_ratio_sp'].iat[0] # 返回一个序列
            return value
        def get_lable(x):
            if x >= 0:
                return True
            else: 
                return False
        input_data['value'] = list(input_data.groupby('start_date').apply(get_value)) # 转为list
        input_data['lable'] = list(input_data['value'].apply(get_lable))
        input_data.loc[:,'value'] = input_data.transform({'value' : lambda x : np.square(x)})
        # input_data.groupby(['model','lable']).sum().apply(get_loss)
        input_filter = input_data.groupby(['model','lable']).agg({'value' : 'sum'})
        input_filter.drop(input_filter.tail(1).index,inplace=True)
        input_filter['loss'] = [input_filter.loc[model].diff()['value'].iat[1] for model in list(input_filter.index.get_level_values(0))]
        input_filter['loss'].nlargest(10)
    signal_input.rolling(window = window).apply()

In [ ]:
# 损失函数

def get_value(x):
    value = x['sharpe_ratio_sp'] - x[x['model'] == 'benchmark']['sharpe_ratio_sp'].iat[0] # 返回一个序列
    return value
def get_lable(x):
    if x >= 0:
        return True
    else: 
        return False
input_data['value'] = list(input_data.groupby('start_date').apply(get_value)) # 转为list
input_data['lable'] = list(input_data['value'].apply(get_lable))
input_data.loc[:,'value'] = input_data.transform({'value' : lambda x : np.square(x)})
# input_data.groupby(['model','lable']).sum().apply(get_loss)
input_filter = input_data.groupby(['model','lable']).agg({'value' : 'sum'})
input_filter.drop(input_filter.tail(1).index,inplace=True)
input_filter['loss'] = [input_filter.loc[model].diff()['value'].iat[1] for model in list(input_filter.index.get_level_values(0))]
list(input_filter['loss'].nlargest(10).index.get_level_values(0).drop_duplicates())


In [ ]:
get_model(input_data)